# Proyecto de pruebas A/B


### 1️ Objetivo

Evaluar si el nuevo sistema de recomendaciones mejora el rendimiento del embudo de conversión:  
`product_page → product_card → purchase`.

### 2️ Hipótesis

* **H₀ (nula):** No hay diferencia significativa entre los grupos A y B en las tasas de conversión del embudo.  
* **H₁ (alternativa):** El grupo B (nuevo embudo) tiene una conversión al menos **10% superior** en cada etapa (`product_page`, `product_card`, `purchase`).

### 3️ Diseño del experimento

* **Grupos:**
  * A — control (embudo actual)
  * B — experimental (nuevo embudo con sistema de recomendación)
* **Audiencia:** 15% de los nuevos usuarios en la UE
* **Periodo:**
  * Inicio: 2020-12-07
  * Cierre de nuevos usuarios: 2020-12-21
  * Fin total: 2021-01-01
* **Duración efectiva de observación:** 14 días por usuario desde la inscripción.

### 4️ Métricas clave

* **Eventos:**
  * `product_page` (vistas de producto)
  * `product_card` (añadir al carrito)
  * `purchase` (compra completada)
* **Indicadores:**
  * Tasa de conversión por etapa
  * Incremento relativo (%) entre grupos
  * Prueba de significancia (z-test o Mann–Whitney según distribución)

### 5️ Criterio de éxito

≥ 10 % de mejora en cada etapa del embudo con significancia estadística (α = 0.05).


In [1]:
# Exploración de los datos: tipos, nulos y duplicados
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from math import sqrt, erf
from scipy.stats import mannwhitneyu


marketing = pd.read_csv(r"C:\Users\pc\OneDrive\Documentos\TripleTen\Proyecto_Sprint 14\Test AB\ab_project_marketing_events_us.csv")
new_users = pd.read_csv(r"C:\Users\pc\OneDrive\Documentos\TripleTen\Proyecto_Sprint 14\Test AB\final_ab_new_users_upd_us.csv")
events = pd.read_csv(r"C:\Users\pc\OneDrive\Documentos\TripleTen\Proyecto_Sprint 14\Test AB\final_ab_events_upd_us.csv")
participants = pd.read_csv(r"C:\Users\pc\OneDrive\Documentos\TripleTen\Proyecto_Sprint 14\Test AB\final_ab_participants_upd_us.csv")

for df in (marketing, new_users, events, participants):
    df.columns = df.columns.str.lower().str.strip()

print("Datasets cargados y columnas estandarizadas.")

Datasets cargados y columnas estandarizadas.


In [ ]:
# Tipos de datos y conversión de fechas
def convert_dates(df):
    for c in df.columns:
        if ("date" in c) or ("dt" in c):
            df[c] = pd.to_datetime(df[c], errors="coerce")
    return df

marketing = convert_dates(marketing)
new_users = convert_dates(new_users)
events = convert_dates(events)

for name, df in {"marketing": marketing, "new_users": new_users, "events": events, "participants": participants}.items():
    print(f"\n[{name}] tipos:\n{df.dtypes}")



[marketing] tipos:
name                 object
regions              object
start_dt     datetime64[ns]
finish_dt    datetime64[ns]
dtype: object

[new_users] tipos:
user_id               object
first_date    datetime64[ns]
region                object
device                object
dtype: object

[events] tipos:
user_id               object
event_dt      datetime64[ns]
event_name            object
details              float64
dtype: object

[participants] tipos:
user_id    object
group      object
ab_test    object
dtype: object


**(Tipos/Fechas):**  
- Las fechas en `marketing.start_dt/finish_dt`, `new_users.first_date`, `events.event_dt` son pasadas a datatime para facilitar su analsis y intepretacion. 
- `events.details` es numérico (importe en compras); otros eventos tienen `details` nulo por diseño se mantienen no es necesario modificar.


In [ ]:
# Nulos y duplicados
for name, df in {"marketing": marketing, "new_users": new_users, "events": events, "participants": participants}.items():
    print(f"\n[{name}] nulos por columna:")
    print(df.isna().sum())
    print(f"[{name}] duplicados (filas completas): {df.duplicated().sum()}")



[marketing] nulos por columna:
name         0
regions      0
start_dt     0
finish_dt    0
dtype: int64
[marketing] duplicados (filas completas): 0

[new_users] nulos por columna:
user_id       0
first_date    0
region        0
device        0
dtype: int64
[new_users] duplicados (filas completas): 0

[events] nulos por columna:
user_id            0
event_dt           0
event_name         0
details       363447
dtype: int64
[events] duplicados (filas completas): 0

[participants] nulos por columna:
user_id    0
group      0
ab_test    0
dtype: int64
[participants] duplicados (filas completas): 0


**(Nulos/Duplicados):**  
- Nulos solo en `events.details` (esperado).  
- 0 duplicados a nivel fila en todos los datasets.


In [ ]:
# Claves y contaminación
if "user_id" in new_users.columns:
    print(f"new_users: duplicados por user_id = {new_users['user_id'].duplicated().sum()}")

if set(["user_id","ab_test","group"]).issubset(participants.columns):
    dup_users = participants.groupby("user_id")["group"].nunique()
    contamination = dup_users[dup_users > 1]
    print(f"participants: usuarios en más de un grupo = {len(contamination)}")
    if len(contamination) > 0:
        print("Ejemplos:")
        print(contamination.head())

if set(["user_id","event_dt","event_name"]).issubset(events.columns):
    dup_events = events.duplicated(subset=["user_id","event_dt","event_name"]).sum()
    print(f"events: duplicados por (user_id, event_dt, event_name) = {dup_events}")


new_users: duplicados por user_id = 0
participants: usuarios en más de un grupo = 441
Ejemplos:
user_id
0082295A41A867B5    2
00E68F103C66C1F7    2
02313B9E82255F47    2
04F2CF340B4F3822    2
051D59BC38C3B3AA    2
Name: group, dtype: int64
events: duplicados por (user_id, event_dt, event_name) = 0


**Interpretación rápida (Asignación/Claves):**  
- No hay duplicados de `user_id` en `new_users`.  
- Tras limpieza ya no hay contaminación entre grupos: 0.  
- Sin duplicados por (`user_id`,`event_dt`,`event_name`) en `events`.


In [ ]:
# Perfil 'details' y rangos de fechas
if "details" in events.columns and "event_name" in events.columns:
    prof = (
        events.assign(is_null=events["details"].isna())
              .groupby("event_name")["is_null"]
              .agg(total="count", nulos="sum")
              .assign(pct_nulos=lambda x: round(x["nulos"]/x["total"]*100, 2))
    )
    print("\nPerfil de 'details' por evento:")
    print(prof)

    pur = events.loc[events["event_name"]=="purchase","details"]
    pur_num = pd.to_numeric(pur, errors="coerce")
    print(f"\n'purchase' con detalles numéricos: {pur_num.notna().sum()} de {len(pur)} "
          f"({round(pur_num.notna().mean()*100,2)}%)")

def profile_datetime_columns(df, name):
    dt_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.datetime64)]
    if not dt_cols:
        print(f"{name}: sin columnas datetime.")
        return
    print(f"\nRango de fechas — {name}")
    for c in dt_cols:
        print(f"- {c}: {df[c].min()} → {df[c].max()} (n_na={df[c].isna().sum()})")

for name, df in {"marketing": marketing, "new_users": new_users, "events": events}.items():
    profile_datetime_columns(df, name)



Perfil de 'details' por evento:
               total   nulos  pct_nulos
event_name                             
login         182465  182465      100.0
product_cart   60120   60120      100.0
product_page  120862  120862      100.0
purchase       60314       0        0.0

'purchase' con detalles numéricos: 60314 de 60314 (100.0%)

Rango de fechas — marketing
- start_dt: 2020-01-25 00:00:00 → 2020-12-30 00:00:00 (n_na=0)
- finish_dt: 2020-02-07 00:00:00 → 2021-01-07 00:00:00 (n_na=0)

Rango de fechas — new_users
- first_date: 2020-12-07 00:00:00 → 2020-12-23 00:00:00 (n_na=0)

Rango de fechas — events
- event_dt: 2020-12-07 00:00:33 → 2020-12-30 23:36:33 (n_na=0)


**Detalles/Fechas:**  
- `purchase.details` 100% numérico por lo tanto se toma válido como **importe**.  
- Eventos en rango **2020-12-07 → 2020-12-30**; el test cerraba el **2021-01-01** faltan ~2 días, solicitar explicacion porque se omitieron estos 2 dias.


In [ ]:
# Validación temporal, audiencia y participantes
TEST_NAME   = "recommender_system_test"
LAUNCH      = pd.Timestamp("2020-12-07")
STOP_INTAKE = pd.Timestamp("2020-12-21")   # última fecha para nuevos usuarios
TEST_END    = pd.Timestamp("2021-01-01")   # fin oficial
WINDOW_DAYS = 14                           # ventana individual de medición
FUNNEL_EVENTS = ["product_page", "product_cart", "purchase"]
print("Parámetros listos.")

Parámetros listos.



Definimos la ventana experimental y los eventos del embudo. La medición se hace por usuario: desde su `first_date` y por **14 días**, con tope en **2021-01-01**.


In [ ]:
# Participantes del test y contaminación
participants_test = participants.query("ab_test == @TEST_NAME").copy()

dup_users = participants_test.groupby("user_id")["group"].nunique()
contaminated_users = dup_users[dup_users > 1].index
participants_clean = participants_test[~participants_test["user_id"].isin(contaminated_users)].copy()

print(f"Total participantes (test): {participants_test['user_id'].nunique()} | "
      f"Contaminados: {len(contaminated_users)} | "
      f"Limpios: {participants_clean['user_id'].nunique()}")


Total participantes (test): 3675 | Contaminados: 0 | Limpios: 3675


Trabajaremos solo con **`participants_clean`** (excluye cualquier usuario en más de un grupo). Esto asegura asignación A/B válida.

In [ ]:
#  Cohorte EU en ventana de enrolamiento
new_users_eu = new_users.query("region == 'EU' and @LAUNCH <= first_date <= @STOP_INTAKE").copy()

# Enrolled = (nuevos EU en ventana) ∩ (asignados A/B limpios)
enrolled = (new_users_eu[["user_id","first_date"]]
            .merge(participants_clean[["user_id","group"]], on="user_id", how="inner"))

print(f"Nuevos EU ventana: {new_users_eu['user_id'].nunique()} | "
      f"Enrolled (EU∩A/B): {enrolled['user_id'].nunique()}")
print("\nBalance por grupo (inscritos):")
print(enrolled.groupby("group")["user_id"].nunique())


Nuevos EU ventana: 39466 | Enrolled (EU∩A/B): 3481

Balance por grupo (inscritos):
group
A    2604
B     877
Name: user_id, dtype: int64



###  Análisis exploratorio (EDA)

**1.- Conversión en las diferentes etapas del embudo**  
El grupo **B** tuvo menores tasas en todas las etapas (`product_page`, `product_cart`, `purchase`).

- No hubo incremento ≥10 % por lo tanto se logró.  
- Las diferencias negativas en “page” (-13 %) y “purchase” (-11 %) fueron **estadísticamente significativas** (p < 0.05).

**2.- ¿El número de eventos por usuario está distribuido equitativamente?**  
No. El grupo **A** tiene 2604 usuarios y el grupo **B** solo 877.  
Esa relación ≈ 3:1 genera **desequilibrio muestral**, reduciendo la potencia de las pruebas.

**3.- ¿Hay usuarios presentes en ambas muestras?**  
No. Tras la limpieza, la contaminación fue **0 usuarios**.  
La asignación A/B esta limpia.

**4.- Distribución de eventos entre los días**  
Los eventos se registraron entre **2020-12-07 y 2020-12-30**, con actividad continua y dentro de la ventana experimental.  
No hay fugas antes del registro ni después del 01-ene-2021 → coherente con el protocolo.

**5.- Peculiaridades antes de iniciar la prueba**
- Muestra menor de lo previsto (3481 vs 6000).
- Cobertura 8.8 % < 15 % planificada.
- Fuerte desbalance A/B.
- Fechas de eventos ligeramente más cortas (faltan 2 días).  
 Todo esto debe documentarse porque afecta la representatividad y significancia.



In [ ]:
# Cobertura de audiencia
den = new_users_eu["user_id"].nunique()
num = enrolled["user_id"].nunique()
coverage = (num / den) if den else np.nan
print(f"Cobertura del test en nuevos EU (ventana): {coverage:.2%}  |  Target ≈ 15%")

# Ventana de 14 días por usuario (cap en TEST_END)
events_ab = events.merge(enrolled[["user_id","first_date","group"]], on="user_id", how="inner")
events_ab["win_end"] = (events_ab["first_date"] + pd.to_timedelta(WINDOW_DAYS, unit="D")).clip(upper=TEST_END)

events_in_scope = events_ab.query("first_date <= event_dt and event_dt < win_end").copy()
events_in_scope = events_in_scope[events_in_scope["event_name"].isin(FUNNEL_EVENTS)]

print(f"Eventos en ventana 14d (cap {TEST_END.date()}): {len(events_in_scope):,}")
print(events_in_scope["event_name"].value_counts())


Cobertura del test en nuevos EU (ventana): 8.82%  |  Target ≈ 15%
Eventos en ventana 14d (cap 2021-01-01): 12,033
product_page    6132
purchase        2998
product_cart    2903
Name: event_name, dtype: int64


In [ ]:
# dataset reducido con tasas
plot_df = results[["metric", "A_rate", "B_rate"]].melt(id_vars="metric", var_name="group", value_name="rate")

plt.figure(figsize=(8,5))
for i, stage in enumerate(plot_df["metric"].unique()):
    subset = plot_df[plot_df["metric"]==stage]
    plt.bar(subset["group"], subset["rate"], label=stage)
    plt.title("Tasas de conversión por grupo y etapa del embudo")
    plt.ylabel("Tasa de conversión")
    plt.legend()
plt.show()

Contrastamos la **cobertura real** vs el **objetivo (15%)**. Coberturas bajas reducen potencia y pueden sesgar representatividad.

Nos quedamos solo con eventos del embudo y **dentro de la ventana** definida por usuario. Esto nos evita “fugas” fuera de la medición para mayor precision de analsis .


In [ ]:
# Chequeos de sanidad
leaks = (events_ab["event_dt"] < events_ab["first_date"]).sum()
print(f"Eventos antes de la inscripción (esperado 0): {leaks}")

print(f"Rango real de events: {events['event_dt'].min()} → {events['event_dt'].max()} | "
      f"Fin planificado: {TEST_END}")

# %% Artefactos listos para embudo y métricas
print("Listo: usa `enrolled` y `events_in_scope` para calcular embudos y conversiones por grupo.")

Eventos antes de la inscripción (esperado 0): 0
Rango real de events: 2020-12-07 00:00:33 → 2020-12-30 23:36:33 | Fin planificado: 2021-01-01 00:00:00
Listo: usa `enrolled` y `events_in_scope` para calcular embudos y conversiones por grupo.


- **Fugas = 0** confirma consistencia temporal.
    
- Si la última fecha real < `TEST_END`, documenta la pequeña diferencia 
(no invalida, pero reduce cobertura de los últimos días).


In [ ]:
# Construir flags por usuario para cada etapa del embudo 

# Embudo base por usuario
FUNNEL_EVENTS = ["product_page", "product_cart", "purchase"]
df = events_in_scope[events_in_scope["event_name"].isin(FUNNEL_EVENTS)].copy()

user_step = (
    df.pivot_table(index="user_id", columns="event_name", values="event_dt",
                   aggfunc="count", fill_value=0)
      .reindex(columns=FUNNEL_EVENTS, fill_value=0)
      .astype(int)
      .rename(columns={"product_page": "n_page", "product_cart": "n_cart", "purchase": "n_purch"})
)

user_step["seen_page"] = (user_step["n_page"] > 0).astype(int)
user_step["add_cart"]  = (user_step["n_cart"] > 0).astype(int)
user_step["purchased"] = (user_step["n_purch"] > 0).astype(int)
print(user_step.head())



event_name        n_page  n_cart  n_purch  seen_page  add_cart  purchased
user_id                                                                  
001064FEAAB631A1       3       0        0          1         0          0
0010A1C096941592       4       0        4          1         0          1
003DF44D7589BBD4       5       5        0          1         1          0
005E096DBD379BCF       0       0        2          0         0          1
006E3E4E232CE760       3       0        0          1         0          0


Creamos variables binarias (`seen_page`, `add_cart`, `purchased`) que indican si el usuario alcanzó cada etapa.  
Esto permite medir tasas de conversión individuales y evitar sesgos por número de eventos.

In [ ]:
# Revenue y dataset final
rev = (
    df.loc[df["event_name"]=="purchase", ["user_id","details"]]
      .groupby("user_id", as_index=False)["details"].sum()
      .rename(columns={"details": "rev_14d"})
)

user_metrics = (
    enrolled[["user_id","group"]]
      .merge(user_step.reset_index(), on="user_id", how="left")
      .merge(rev, on="user_id", how="left")
      .fillna(0)
)
print(user_metrics.head())


            user_id group  n_page  n_cart  n_purch  seen_page  add_cart  \
0  D72A72121175D8BE     A     1.0     0.0      0.0        1.0       0.0   
1  DD4352CDCF8C3D57     B     5.0     0.0      0.0        1.0       0.0   
2  831887FE7F2D6CBA     A     0.0     3.0      2.0        0.0       1.0   
3  4CB179C7F847320B     B     3.0     0.0      0.0        1.0       0.0   
4  3C5DD0288AC4FE23     A     1.0     0.0      1.0        1.0       0.0   

   purchased  rev_14d  
0        0.0     0.00  
1        0.0     0.00  
2        1.0   104.98  
3        0.0     0.00  
4        1.0     4.99  


`user_metrics` contiene métricas completas por usuario: grupo A/B, pasos del embudo y `rev_14d` (suma de compras).
Pasaremos a calcular tasas y comparaciones estadísticas.

In [ ]:
# Funciones auxiliares

def prop_ci(p, n, z=1.96):
    se = np.sqrt(p*(1-p)/n)
    return (p - z*se, p + z*se)

def two_prop_ztest(x1, n1, x2, n2):
    p_pool = (x1 + x2) / (n1 + n2)
    se = np.sqrt(p_pool*(1 - p_pool)*(1/n1 + 1/n2))
    z = (x2/n2 - x1/n1) / se
    Phi = lambda t: 0.5*(1 + erf(t/sqrt(2)))
    p = 2*(1 - Phi(abs(z)))
    return z, p

# Tasas y prueba z
def summarize_conversion(col, denom_filter=None):
    data = user_metrics.copy()
    if denom_filter is not None:
        data = data[denom_filter(data)]
    g = data.groupby("group")[col].agg(["sum","count"])
    a, b = g.loc["A"], g.loc["B"]
    pA, pB = a["sum"]/a["count"], b["sum"]/b["count"]
    z, p = two_prop_ztest(a["sum"], a["count"], b["sum"], b["count"])
    lift = (pB - pA)/pA
    return pd.DataFrame({
        "metric": [col],
        "A_rate": [pA], "B_rate": [pB],
        "lift_B_vs_A": [lift],
        "z_stat": [z], "p_value": [p],
        "A_n": [a["count"]], "B_n": [b["count"]]
    })

results = pd.concat([
    summarize_conversion("seen_page"),
    summarize_conversion("add_cart"),
    summarize_conversion("purchased"),
    summarize_conversion("add_cart", lambda d: d["seen_page"]==1),
    summarize_conversion("purchased", lambda d: d["add_cart"]==1),
    summarize_conversion("purchased", lambda d: d["seen_page"]==1)
])
print(results)


      metric    A_rate    B_rate  lift_B_vs_A    z_stat   p_value     A_n  \
0  seen_page  0.647081  0.562144    -0.131263 -4.495436  0.000007  2604.0   
0   add_cart  0.300307  0.278221    -0.073545 -1.240767  0.214692  2604.0   
0  purchased  0.319892  0.283922    -0.112444 -1.990600  0.046525  2604.0   
0   add_cart  0.302671  0.275862    -0.088573 -1.146252  0.251691  1685.0   
0  purchased  0.333760  0.307377    -0.079047 -0.766645  0.443292   782.0   
0  purchased  0.343620  0.314402    -0.085032 -1.207201  0.227355  1685.0   

     B_n  
0  877.0  
0  877.0  
0  877.0  
0  493.0  
0  244.0  
0  493.0  


**Interpretación:**  
Definimos funciones para calcular **intervalos de confianza**, **prueba z de dos proporciones** y usaremos `mannwhitneyu` para revenue.
  
Cada fila muestra la tasa de conversión, el **lift** del grupo B frente a A, y su **p-value** de la prueba z.  
Esto permite identificar qué etapas presentan diferencias estadísticamente significativas.


In [ ]:
# Métricas de revenue
rev_stats = (
    user_metrics.groupby("group")["rev_14d"]
      .agg(n="count", mean="mean", median="median",
           p90=lambda s: s.quantile(0.9),
           purchasers=lambda s: (s>0).sum())
)
rev_stats["purch_rate"] = rev_stats["purchasers"]/rev_stats["n"]

aov = (
    user_metrics[user_metrics["rev_14d"]>0]
      .groupby("group")["rev_14d"]
      .agg(mean="mean", median="median", p90=lambda s: s.quantile(0.9))
)

u_stat, p_u = mannwhitneyu(
    user_metrics.loc[user_metrics["group"]=="A","rev_14d"],
    user_metrics.loc[user_metrics["group"]=="B","rev_14d"],
    alternative="two-sided"
)
print("ARPU / AOV / Mann-Whitney p:", p_u)


ARPU / AOV / Mann-Whitney p: 0.005288962672564268



**Interpretación:**  
- **ARPU** (ingreso promedio por usuario) incluye ceros; mide impacto general.  
- **AOV** (promedio de compra) mide ticket medio solo de compradores.  
- **Mann-Whitney** evalúa diferencia en distribución de ingresos (robusto ante muchos ceros).  
Un **p < 0.05** sugiere diferencia significativa.



## Resultados y Conclusiones
###  Evaluación de la prueba A/B

**1.- Resultados del test**  
El nuevo sistema **no mejoró la conversión**.

- Menor proporción de vistas, las compras y el ARPU.  
- p-values < 0.05 en “purchase” y “ARPU” indican diferencias **estadísticamente significativas** en perjuicio del grupo B.  
 El sistema de recomendaciones **empeora** el rendimiento comercial.

**2.- Prueba z aplicada** 

Ya fue usada en tu tabla (`z_stat` y `p_value`):

- `purchase`: z = −1.99, p = 0.046 → significativa (rechaza H₀).  
- Otras etapas: p > 0.05 → sin diferencia estadística.


###  Conclusiones generales

- **EDA** confirma que los datos estan limpios, sin duplicados ni contaminación, pero con desbalance y una menor muestra.  
- **Prueba A/B** muestra que el grupo B tiene peor desempeño en conversión y revenue.  
- **Decisión**: no implementar el nuevo embudo; se recomienda analizar causas (interfaz, recomendaciones o fricción en pago).

